In [1]:
import pvdeg
import inspire_agrivolt

import xarray as xr
import numpy as np
import pandas as pd

from dask.distributed import Client, LocalCluster
import dask

dask.config.set({"array.slicing.split_large_chunks": True})

In [2]:
cluster = LocalCluster(
    n_workers=8,        # Number of processes (can set to number of CPU cores you want to use)
)

client = Client(cluster)

client

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 44525 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:44525/status,
Dashboard: http://127.0.0.1:44525/status,Workers: 8
Total threads: 128,Total memory: 503.15 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45753,Workers: 8
Dashboard: http://127.0.0.1:44525/status,Total threads: 128
Started: Just now,Total memory: 503.15 GiB
Comm: tcp://127.0.0.1:39869,Total threads: 16
Dashboard: http://127.0.0.1:36179/status,Memory: 62.89 GiB
Nanny: tcp://127.0.0.1:37241,


In [ ]:
store = xr.open_zarr("https://oedi-data-lake.s3.amazonaws.com/inspire/agrivoltaics_irradiance/test-all-states/01.zarr/")

In [3]:
geo_weather, geo_meta = inspire_agrivolt.irradiance_sam._load_full_nsrdb_kestrel()

usa_meta = geo_meta[geo_meta["country"] == "United States"]
usa_weather = geo_weather.sel(gid=usa_meta.index.values)

/home/tford/.conda-envs/geospatial/lib/python3.9/site-packages/xarray/core/dataset.py:277: UserWarning: The specified chunks separate the stored chunks along dimension "phony_dim_1" starting at index 500. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


In [19]:
import dask.array as da

weather_df = pd.DataFrame(columns=["temp_air", "wind_speed", "wind_direction", "dhi", "ghi", "dni", "relative_humidity", "albedo"], data=np.zeros((8760, 8)), index=pd.date_range(start="2001-01-01", periods=8760, freq="1h"))
timeseries = np.zeros((8760, ))
ground_irradiance = np.zeros((8761, 11))

res_dict = {
    "annual_poa_front":0,
    "annual_energy":0,
    "subarray1_poa_front":timeseries, 
    "subarray1_poa_rear":timeseries,
    "subarray1_celltemp":timeseries,
    "subarray1_dc_gross":timeseries,
    "temp_air":timeseries, 
    "wind_speed":timeseries, 
    "wind_direction":timeseries, 
    "dhi":timeseries, 
    "ghi":timeseries, 
    "dni":timeseries, 
    "relative_humidity":timeseries, 
    "albedo":timeseries,
    "subarray1_ground_rear_spatial":ground_irradiance
}

client.scatter(res_dict)
client.scatter(


def caller(weather_df, meta, res_dict):
    return pvdeg.pysam._handle_pysam_return(res_dict, weather_df, 0, 0)


<xarray.Dataset> Size: 2MB
Dimensions:              (time: 8760, distance: 10)
Coordinates:
  * time                 (time) datetime64[ns] 70kB 2001-01-01 ... 2001-12-31...
  * distance             (distance) int64 80B 0 1 2 3 4 5 6 7 8 9
Data variables: (12/17)
    tilt                 float64 8B 0.0
    pitch                float64 8B 0.0
    annual_poa           int64 8B 0
    annual_energy        int64 8B 0
    subarray1_poa_front  (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    subarray1_poa_rear   (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    ...                   ...
    dhi                  (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    ghi                  (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    dni                  (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    relative_humidity    (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    albedo               (time) float64 70kB dask.array<chunksize=(8760,), meta=np.ndarray>
    ground_irradiance    (time, distance) float64 701kB dask.array<chunksize=(8760, 10), meta=np.ndarray>

In [ ]:
pvdeg.geospatial.analysis(
    usa_weather.isel(gid=slice(0,10)),
    usa_meta.iloc[0:10],
    
)

In [10]:
timeseries = np.zeros((8760, ))

timeseries

array([0., 0., 0., ..., 0., 0., 0.])

In [ ]:
stacked = pvdeg.geospatial.output_template(
    usa_weather,
    pvdeg.pysam.INSPIRE_GEOSPATIAL_TEMPLATE_SHAPES,
    add_dims={"distance":10}
)

stacked = stacked.drop(["gid"])

mindex_obj = pd.MultiIndex.from_arrays(
    [meta_df["latitude"], meta_df["longitude"]], names=["latitude", "longitude"]
)
mindex_coords = xr.Coordinates.from_pandas_multiindex(mindex_obj, "gid")
stacked = stacked.assign_coords(mindex_coords)

# stacked = stacked.drop_duplicates("gid")
# res = stacked.unstack("gid")  # , sparse=True

stacked

In [ ]:
stacked_by = stacked.groupby("gid").first()

stacked_by

In [ ]:
for val in stacked.latitude.values[:5]:  # Just to preview a few
    print(f"{val:.6f}")

print()

for val in stacked.longitude.values[:5]:  # Just to preview a few
    print(f"{val:.6f}")


In [ ]:
gb = 0
for var in res.data_vars:
    gb += res[var].nbytes  * 1e-9

# 3.3 TB
gb

In [ ]:
futures = []

for 

In [ ]:
res.to_zarr("/projects/inspire/PySAM-MAPS/test-big-zarr.zarr", write_empty_chunks=True)